In [1]:
from dotenv import load_dotenv
load_dotenv("/home/marshath/play/chainlink/algovate/.env")

Python-dotenv could not parse statement starting at line 4


True

In [7]:
import pandas as pd
from algovate.autoq.autoq import AutoQ

In [11]:
import time
import pickle
import pandas as pd
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import display, Markdown

from llama_index import Document
from llama_index.node_parser import SimpleNodeParser
from llama_index import GPTVectorStoreIndex, ResponseSynthesizer
from llama_index.data_structs.node import Node, DocumentRelationship

from langchain.chat_models import ChatOpenAI
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index import LLMPredictor, GPTVectorStoreIndex, PromptHelper, ServiceContext
from llama_index import GPTListIndex
from llama_index.indices.list.retrievers import ListIndexLLMRetriever
from llama_index.indices.postprocessor import (
    LLMRerank
)
from llama_index.indices.postprocessor.node import AutoPrevNextNodePostprocessor

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.indices.postprocessor.node import PrevNextNodePostprocessor
from llama_index.node_parser import SimpleNodeParser
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.storage_context import StorageContext

from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

In [12]:
with open('/home/marshath/play/chainlink/algovate/algovate/data/combined_documents.pkl', 'rb') as f:
    cl_combineddoc = pickle.load(f)

# convert to llama documents
llama_cl_combineddoc = []
for doc in cl_combineddoc:
    llama_cl_combineddoc.append(Document(text=doc.page_content, extra_info=doc.metadata))

In [13]:
def retrieve_docs(df, query_engine):
    # Add retrieved_docs column if it doesn't exist
    df['retrieved_docs'] = df.get('retrieved_docs', '')

    for index, row in df.iterrows():
        retrieved_docs = query_engine.retrieve(row['question'])

        docs = ""
        for d in retrieved_docs:
            if docs == "":
                docs = d.node.text
            else:
                docs = docs + "\n\n:::NEXT DOC:::\n\n" + d.node.text

        # Store retrieved docs in the DataFrame
        df.at[index, 'retrieved_docs'] = docs

In [14]:
def evaluate_dataframe_answer(df):
    # Add right_or_wrong column if it doesn't exist
    if 'right_or_wrong' not in df.columns:
        df['right_or_wrong'] = ''

    # Widgets
    button_right = widgets.Button(description="Right")
    button_wrong = widgets.Button(description="Wrong")
    button_partial = widgets.Button(description="partial")
    out = widgets.Output()

    # Starting index
    index = [0]  # Use list to allow mutation inside functions

    def display_row(row):
        """
        Display the row data in markdown
        """
        with out:
            display(Markdown(f"**Question**: {row['question']}"))
            display(Markdown(f"**Answer**: {row['answer']}"))
            display(Markdown(f"**Result**: {row['result']}\n"))

    def on_button_right_clicked(b):
        """
        Update right_or_wrong column with 'right' and go to next row
        """
        df.at[index[0], 'right_or_wrong'] = 'right'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df):
            display_row(df.iloc[index[0]])

    def on_button_wrong_clicked(b):
        """
        Update right_or_wrong column with 'wrong' and go to next row
        """
        df.at[index[0], 'right_or_wrong'] = 'wrong'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df):
            display_row(df.iloc[index[0]])

    def on_button_partial_clicked(b):
        """
        Update right_or_wrong column with 'partial' and go to next row
        """
        df.at[index[0], 'right_or_wrong'] = 'partial'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df):
            display_row(df.iloc[index[0]])

    button_right.on_click(on_button_right_clicked)
    button_wrong.on_click(on_button_wrong_clicked)
    button_partial.on_click(on_button_partial_clicked)

    display(out)
    display(button_right, button_wrong, button_partial)

    # Start by displaying the first row
    display_row(df.iloc[0])

In [15]:
def evaluate_dataframe_retrieval(df):
    # Add rd_right_or_wrong column if it doesn't exist
    df['rd_right_or_wrong'] = df.get('rd_right_or_wrong', '')

    df_wrong = df[df['right_or_wrong'].isin(['wrong', 'partial'])].copy()

    # Widgets
    button_right = widgets.Button(description="Right")
    button_wrong = widgets.Button(description="Wrong")
    button_partial = widgets.Button(description="Partial")
    out = widgets.Output()

    # Starting index
    index = [0]  # Use list to allow mutation inside functions

    def display_row(row):
        """
        Display the row data in markdown
        """
        with out:
            display(Markdown(f"**Question**: {row['question']}"))
            display(Markdown(f"**Answer**: {row['answer']}"))
            display(Markdown(f"**Result**: {row['result']}\n"))
            display(Markdown(f"**Retrieved Docs**: {row['retrieved_docs']}"))

    def on_button_right_clicked(b):
        """
        Update right_or_wrong column with 'right' and go to next row
        """
        df.at[df_wrong.index[index[0]], 'rd_right_or_wrong'] = 'right'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df_wrong):
            display_row(df_wrong.iloc[index[0]])

    def on_button_wrong_clicked(b):
        """
        Update right_or_wrong column with 'wrong' and go to next row
        """
        df.at[df_wrong.index[index[0]], 'rd_right_or_wrong'] = 'wrong'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df_wrong):
            display_row(df_wrong.iloc[index[0]])

    def on_button_partial_clicked(b):
        """
        Update right_or_wrong column with 'partial' and go to next row
        """
        df.at[df_wrong.index[index[0]], 'rd_right_or_wrong'] = 'partial'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df_wrong):
            display_row(df_wrong.iloc[index[0]])

    button_right.on_click(on_button_right_clicked)
    button_wrong.on_click(on_button_wrong_clicked)
    button_partial.on_click(on_button_partial_clicked)

    display(out)
    display(button_right, button_wrong, button_partial)

    # Start by displaying the first row
    if len(df_wrong) > 0:
        display_row(df_wrong.iloc[0])
    else:
        print("There are no wrong rows in the DataFrame.")

In [17]:
# this dataframe is the output of the below AutoQ block
df = pd.read_csv("/home/marshath/play/chainlink/algovate/algovate/data/auto_eval_pairs.csv")

### AutoQ

In [3]:
autoq = AutoQ(document_store="/home/marshath/play/chainlink/algovate/algovate/data/combined_documents.pkl")

AutoQ initialized
Document store has [Document(page_content="## Welcome to the Chainlink Documentation\n\n##### New to Chainlink and Smart Contracts?\n\nLearn the basics in the Getting Started Guide.\n\n## Explore our products\n\n#### [Data Feeds](data-feeds)\n\nDecentralized and high-quality data feeds for DeFi, Reserves, NFTs, sports,\nweather, and more\n\n###### Available on:\n\n#### [Chainlink Functions](chainlink-functions)\n\nConnect smart contracts to a trust-minimized compute infrastructure running on\na decentralized oracle network\n\n###### Available on:\n\n#### [Automate Contracts](chainlink-automation/introduction)\n\nDecentralized, highly reliable, cost-efficient and highly secure automation\nfor smart contracts\n\n###### Available on:\n\n#### [VRF v2](/vrf/v2/introduction)\n\nVerifiable, tamper-proof random number generator for blockchain gaming and NFT\nprojects\n\n###### Available on:\n\n#### [External API Calls](/any-api/introduction)\n\nRequest and receive data from a

In [4]:
autoq.generate_eval_pairs(50)

Generating 50 pairs
Generated 1 of 50 pairs
Error generating pair for chunk 
Generated 2 of 50 pairs
Generated 3 of 50 pairs
Error generating pair for chunk 
Generated 4 of 50 pairs
Generated 5 of 50 pairs
Generated 6 of 50 pairs
Generated 7 of 50 pairs
Generated 8 of 50 pairs
Generated 9 of 50 pairs
Generated 10 of 50 pairs
Generated 11 of 50 pairs
Generated 12 of 50 pairs
Generated 13 of 50 pairs
Error generating pair for chunk nderlying aggregator | aggregator         | aggregator.eth-usd.data.eth |
| Proposed aggregator   | proposed           | proposed.eth-usd.data.eth   |
## Architecture

### Resolver

For each network, there is a single Chainlink resolver, which does not change.
Its address can be obtained using the `data.eth` domain. This resolver manages
the subdomains associated with `data.eth`.

| Network          | Resolver Address                           |
|:-----------------|:-------------------------------------------|
| Ethereum Mainnet | 0x122eb74f9d0F1a5ed587F43D120

In [9]:
pd.DataFrame(autoq.pairs).to_csv("/home/marshath/play/chainlink/algovate/algovate/data/auto_eval_pairs.csv", index=False)

### Base case

In [18]:
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0,))

max_input_size = 4096
num_output = 256
max_chunk_overlap = 20

prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

In [19]:
index_base_combineddoc = GPTVectorStoreIndex.from_documents(
    llama_cl_combineddoc, 
    service_context=service_context
)

query_engine_base_combineddoc = index_base_combineddoc.as_query_engine()

In [20]:
df2 = df.copy()
for i, gt in df2.iterrows():
    try:
        df2.loc[i, "result"] = query_engine_base_combineddoc.query(gt.question)
    except Exception as e:
        print(e)
        df2.loc[i, "result"] = "ERROR"

    print(f"Question {i+1} of {len(df2)}")

Question 1 of 50
Question 2 of 50
Question 3 of 50
Question 4 of 50
Question 5 of 50
Question 6 of 50
Question 7 of 50
Question 8 of 50
Question 9 of 50
Question 10 of 50
Question 11 of 50
Question 12 of 50
Question 13 of 50
Question 14 of 50
Question 15 of 50
Question 16 of 50
Question 17 of 50
Question 18 of 50
Question 19 of 50
Question 20 of 50
Question 21 of 50
Question 22 of 50
Question 23 of 50
Question 24 of 50
Question 25 of 50
Question 26 of 50
Question 27 of 50
Question 28 of 50
Question 29 of 50
Question 30 of 50
Question 31 of 50
Question 32 of 50
Question 33 of 50
Question 34 of 50
Question 35 of 50
Question 36 of 50
Question 37 of 50
Question 38 of 50
Question 39 of 50
Question 40 of 50
Question 41 of 50
Question 42 of 50
Question 43 of 50
Question 44 of 50
Question 45 of 50
Question 46 of 50
Question 47 of 50
Question 48 of 50
Question 49 of 50
Question 50 of 50


In [23]:
retrieve_docs(df2, query_engine_base_combineddoc)
df2.to_csv("/home/marshath/play/chainlink/algovate/test_chainlink/autoq_combineddoc_base.csv", index=False)

### LLMReranker

In [25]:
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, chunk_size_limit=512)
reranker = LLMRerank(choice_batch_size=5, top_n=3, service_context=service_context)

index_llmreranker_combineddoc = GPTVectorStoreIndex.from_documents(llama_cl_combineddoc, service_context=service_context)

query_engine_llmreranker_combineddoc = index_llmreranker_combineddoc.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[reranker],
)

In [26]:
df3 = df.copy()
for i, gt in df3.iterrows():
    try:
        df3.loc[i, "result"] = query_engine_base_combineddoc.query(gt.question)
    except Exception as e:
        print(e)
        df3.loc[i, "result"] = "ERROR"

    print(f"Question {i+1} of {len(df3)}")

Question 1 of 50
Question 2 of 50
Question 3 of 50
Question 4 of 50
Question 5 of 50
Question 6 of 50
Question 7 of 50
Question 8 of 50
Question 9 of 50
Question 10 of 50
Question 11 of 50
Question 12 of 50
Question 13 of 50
Question 14 of 50
Question 15 of 50
Question 16 of 50
Question 17 of 50
Question 18 of 50
Question 19 of 50
Question 20 of 50
Question 21 of 50
Question 22 of 50
Question 23 of 50
Question 24 of 50
Question 25 of 50
Question 26 of 50
Question 27 of 50
Question 28 of 50
Question 29 of 50
Question 30 of 50
Question 31 of 50
Question 32 of 50
Question 33 of 50
Question 34 of 50


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 7d1f7320486573905a92f3ac4a38fb73 in your message.).


Question 35 of 50
Question 36 of 50
Question 37 of 50
Question 38 of 50
Question 39 of 50
Question 40 of 50
Question 41 of 50
Question 42 of 50
Question 43 of 50
Question 44 of 50
Question 45 of 50
Question 46 of 50
Question 47 of 50
Question 48 of 50
Question 49 of 50
Question 50 of 50


In [27]:
retrieve_docs(df3, query_engine_llmreranker_combineddoc)

In [28]:
df3.to_csv("/home/marshath/play/chainlink/algovate/test_chainlink/autoq_combineddoc_llmreranker.csv", index=False)

### AutoPrevNext

In [29]:
service_context = ServiceContext.from_defaults()

nodes = service_context.node_parser.get_nodes_from_documents(llama_cl_combineddoc)
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)
storage_context = StorageContext.from_defaults(docstore=docstore)

index_autopn_combineddoc = GPTVectorStoreIndex(nodes, storage_context=storage_context)

node_postprocessor = AutoPrevNextNodePostprocessor(docstore=docstore, num_nodes=4, service_context=service_context)

query_engine_autopn_combineddoc = index_autopn_combineddoc.as_query_engine(
    similarity_top_k=2,
    node_postprocessors=[node_postprocessor],
    response_mode="tree_summarize"
)

In [30]:
df4 = df.copy()
for i, gt in df4.iterrows():
    try:
        df4.loc[i, "result"] = query_engine_base_combineddoc.query(gt.question)
    except Exception as e:
        print(e)
        df4.loc[i, "result"] = "ERROR"

    print(f"Question {i+1} of {len(df4)}")

Question 1 of 50
Question 2 of 50
Question 3 of 50
Question 4 of 50
Question 5 of 50
Question 6 of 50
Question 7 of 50
Question 8 of 50
Question 9 of 50
Question 10 of 50
Question 11 of 50
Question 12 of 50
Question 13 of 50
Question 14 of 50
Question 15 of 50
Question 16 of 50
Question 17 of 50
Question 18 of 50
Question 19 of 50
Question 20 of 50
Question 21 of 50
Question 22 of 50
Question 23 of 50
Question 24 of 50
Question 25 of 50
Question 26 of 50
Question 27 of 50
Question 28 of 50
Question 29 of 50
Question 30 of 50
Question 31 of 50
Question 32 of 50
Question 33 of 50
Question 34 of 50
Question 35 of 50
Question 36 of 50
Question 37 of 50
Question 38 of 50
Question 39 of 50
Question 40 of 50
Question 41 of 50
Question 42 of 50
Question 43 of 50
Question 44 of 50
Question 45 of 50
Question 46 of 50
Question 47 of 50
Question 48 of 50
Question 49 of 50
Question 50 of 50


In [31]:
retrieve_docs(df4, query_engine_autopn_combineddoc)

In [32]:
df4.to_csv("/home/marshath/play/chainlink/algovate/test_chainlink/autoq_combineddoc_autopn.csv", index=False)

In [33]:
df2

,question,answer,metadata,result,retrieved_docs
0,What is the purpose of hosting encrypted secre...,To replace the default off-chain option and se...,{'source': 'https://docs.chain.link/chainlink-...,The purpose of hosting encrypted secrets on AW...,# Using Off-chain Secrets in Requests\n\n * O...
1,What was the error message received when runni...,opening db: failed to open db: failed to conne...,{'source': 'https://stackoverflow.com/question...,"For the first context, the error message recei...",at resolveAddresses (C:\Users\Coding\vs...
2,What is the Monte Carlo fallacy?,The Monte Carlo fallacy is the mistaken belief...,{'source': 'https://github.com/oceanByte/chain...,The Monte Carlo fallacy describes the mistaken...,#####Chapter 1:\n\n# Introduction\n\n<ContentW...
3,What was the error message received while exec...,Gas estimation errored with the following mess...,{'source': 'https://stackoverflow.com/question...,The error message received while executing the...,Question: (Asked on: 2023-01-31 17:44:37)\nUna...
4,What is the purpose of the VRFv2SubscriptionMa...,The purpose of the VRFv2SubscriptionManager co...,{'source': 'https://github.com/oceanByte/chain...,The purpose of the VRFv2SubscriptionManager co...,#####Chapter 5: VRF Subscriptions\n\n# Subscri...
5,What is the correct way to mock a response usi...,The _fulfillOracleRequest_ function takes a by...,{'source': 'https://stackoverflow.com/question...,The correct way to mock a response using the _...,Question: (Asked on: 2021-12-18 18:10:26)\nHow...
6,What are events in Solidity and how can they b...,Events in Solidity are a way to emit events th...,{'source': 'https://github.com/oceanByte/chain...,Events in Solidity are a way to emit events th...,#####Chapter 7:\n\n# Events\n\nWhen learning a...
7,What is an array in programming?,An array is many related pieces of data stored...,{'source': 'https://github.com/oceanByte/chain...,An array in programming is a variable that sto...,#####Chapter 8:\n\n# Creating Arrays\n\n<!-- <...
8,What is the cryptocurrency symbol for Cardano?,ADA,{'source': 'https://docs.chain.link/data-feeds...,The cryptocurrency symbol for Cardano is ADA.,|\n| 🟢ADA / USD | C...
9,What are the three different locations to stor...,The three different locations to store variabl...,{'source': 'https://github.com/oceanByte/chain...,The three different locations to store variabl...,"Variables can be declared as storage, memory o..."
